In [1]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
#importing libs
import sklearn
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn import metrics
print(sklearn.__version__)
from nltk.corpus import stopwords
import re
import string
from nltk.stem import WordNetLemmatizer
import itertools

1.2.2


In [3]:
#reading the data
dataset = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/CRUx/Task 3/Dataset.csv")
dataset = shuffle(dataset, random_state = 42)
#print(dataset.columns.tolist())
convert_dict = {'id': str,
                'title': str,
                'Category': int
                }
dataset = dataset.astype(convert_dict)
X_train_non, X_test_non, y_train, y_test = train_test_split(dataset['title'], dataset['Category'])
X_train_non.to_csv('Predict w saved model.csv')
X_train_non = np.array(X_train_non)
X_test_non = np.array(X_test_non)
y_train = np.array(y_train)
y_test = np.array(y_test)

#dataset

In [4]:
X_train_non.shape

(5535,)

In [5]:
dataset['Category'].value_counts()

0    1485
3    1480
4    1479
1    1477
2    1459
Name: Category, dtype: int64

In [6]:
dataset

,id,title,Category
1103,dmrW9o2GlGA,how to make a paper gun,0
2338,ZMqhjKRUGsY,the kid laroi diva ft lil tecca directed by c...,1
7200,mONA__pZMWw,top 10 sitcoms you forgot were hilarious,4
4013,If0po9MNj9U,day 1 gbr vs jpn astana 2023 world para ice ...,2
2684,Rd9LGGTj_to,best of nora fatehi 2023 jukebox non stop to...,1
...,...,...,...
5191,4QLuElkMXXY,top 10 pc games on mobile shorts ytshorts gta5,3
5226,tqAxR1sVfFw,top 10 new mobile games just released ios and...,3
5390,mFdg2hY8F8M,your pc isnt ready for hogwarts legacy,3
860,m_Aa-JCh68Q,jee main 2024 block strategy 250 sure in 90 d...,0


In [7]:
#initialising stopwords and lematization
stopword_list = stopwords.words('english')
lemmatizer = WordNetLemmatizer()

In [8]:
X_train, X_test = [], []
#pre-processing the title
for i in range(0, len(X_train_non)):
  #print(X_train_non[i])
  _title = re.sub(r'[^a-zA-Z]', ' ', X_train_non[i])
  _title = _title.lower()
  _title = _title.split()
  _title = [lemmatizer.lemmatize(word) for word in _title if not word in set(stopword_list)]
  _title = ' '.join(_title)
  X_train.append(_title)

for i in range(0, len(X_test_non)):
  _title = re.sub(r'[^a-zA-Z]', ' ', X_test_non[i])
  _title = _title.lower()
  _title = _title.split()
  _title = [lemmatizer.lemmatize(word) for word in _title if not word in set(stopword_list)]
  _title = ' '.join(_title)
  X_test.append(_title)

X_train_fit = pd.DataFrame(X_test, columns=['title'])
X_train_fit.to_csv('Predict w saved model.csv')

In [10]:
#initialising tfidf vectorizer
import joblib
tf_idf = TfidfVectorizer(stop_words='english', max_df=0.7)
tfidf_train = tf_idf.fit_transform(X_train)
joblib.dump(tf_idf, 'tfidf_multi.pkl')
tfidf_test = tf_idf.transform(X_test)

In [11]:
#PAC classifier
pac=PassiveAggressiveClassifier(max_iter=50)
pac.fit(tfidf_train,y_train)

#predicted y
y_pred = pac.predict(tfidf_test)

print(metrics.classification_report(y_test, y_pred, target_names=['Educational', 'Music',"Sports","Gaming","Movies"]))

              precision    recall  f1-score   support

 Educational       0.91      0.90      0.91       377
       Music       0.97      0.96      0.97       360
      Sports       0.90      0.92      0.91       347
      Gaming       0.93      0.95      0.94       376
      Movies       0.95      0.92      0.93       385

    accuracy                           0.93      1845
   macro avg       0.93      0.93      0.93      1845
weighted avg       0.93      0.93      0.93      1845



In [13]:
import pickle
filename = '/content/drive/MyDrive/Colab Notebooks/CRUx/Task 3/Models/PAC_Multi_Data_model_1.sav'
pickle.dump(pac, open(filename, 'wb'))

#This next part is for doing test prediction

In [14]:
#test prediction
test_data = ['how to play basketball like a pro']
_test = re.sub(r'[^a-zA-Z]', ' ', test_data[0])
_test = _test.lower()
_test = _test.split()
_test = [lemmatizer.lemmatize(word) for word in _test if not word in set(stopword_list)]
text_processed = [' '.join(_test)]
print(text_processed)

['play basketball like pro']


In [15]:
test_input = tf_idf.transform(text_processed)
test_input.shape

(1, 7540)

In [16]:
result_probs = pac._predict_proba_lr(test_input[0]) #list type
result = pac.predict(test_input[0])
print(result_probs)

if result_probs[0][result][0]>=0.65:
  if result == 0:
    print("Educational", result_probs[0][result][0]*100,"% confidence")
  elif result == 1:
    print("Music", result_probs[0][result][0]*100,"% confidence")
  elif result == 2:
    print("Sports", result_probs[0][result][0]*100,"% confidence")
  elif result == 3:
    print("Gaming", result_probs[0][result][0]*100,"% confidence")
  elif result == 4:
    print("Movies", result_probs[0][result][0]*100,"% confidence")
else:
  print("Unable to classify")
  print(result, result_probs[0][result][0])

[[0.04602434 0.1134282  0.52836324 0.19376482 0.11841939]]
Unable to classify
[2] 0.5283632404731036
